## In this notebook, We'll see about Multi searhch Agent RAG. 
This is  RAG which can fetch data from multiple sources like **arxiv**, **wiki**, **PDFs**, and LLM uses this data to give a final response.

Different Components to learn in this:

1. Tool (Langchain toolkit)
2. Agents in RAG

In [1]:
## Creating a wrapper on top of wikipedia

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max= 200)  ## 200 is the number of characters here 
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)

In [15]:
wiki.name  ## it will return wikipedia

'wikipedia'

In [8]:
### Now, we will be fetching data from a website (like scraping)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
document  = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(document, OllamaEmbeddings(model ="all-minilm"))
retriever = vectordb.as_retriever()
retriever

/tmp/ipykernel_1087309/3366568754.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(document, OllamaEmbeddings(model ="all-minilm"))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x72f0727216f0>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, 'langsmith_search',
                      " Search for information about LangSmith. For any question about Langsmith, you must use this tool.")


In [10]:
retriever_tool.name

'langsmith_search'

In [13]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper)

arxiv.name

'arxiv'

In [18]:
tools = [wiki, arxiv, retriever_tool]


In [33]:
from dotenv import load_dotenv
import os
# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama


llm = ChatOllama(model = "gemma3:1b")



In [34]:
## Now we need to query from this list of tools
## We will use some 'agents' for this. 

from langchain import hub

## Get the prompt to use - you can modify this
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


/home/dheeraj/anaconda3/envs/genai/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [35]:

## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [36]:
### Agnt executor
from langchain.agents import AgentExecutor
agent_executor  = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [38]:
agent_executor.invoke({"input" : "Tell me about Machine Learning"})





> Entering new AgentExecutor chain...
Okay, let's dive into Machine Learning! It’s a really exciting and rapidly growing field, and it’s becoming increasingly important in our lives. Here's a breakdown of what it is, how it works, and why it's significant:

**1. What is Machine Learning?**

At its core, Machine Learning (ML) is about **giving computers the ability to learn from data without being explicitly programmed.** Instead of writing detailed rules for every situation, we feed the computer a lot of data, and it figures out the patterns and relationships on its own.  Think of it as teaching a computer to “think” like a human, but with a lot more mathematical precision.

**2. Key Concepts & Types**

* **Data:** This is the fuel for machine learning. The more high-quality data you have, the better the model will perform.
* **Algorithms:** These are the formulas and processes that allow the computer to learn from the data. There are many different types of algorithms, each suited f

{'input': 'Tell me about Machine Learning',
 'output': "Okay, let's dive into Machine Learning! It’s a really exciting and rapidly growing field, and it’s becoming increasingly important in our lives. Here's a breakdown of what it is, how it works, and why it's significant:\n\n**1. What is Machine Learning?**\n\nAt its core, Machine Learning (ML) is about **giving computers the ability to learn from data without being explicitly programmed.** Instead of writing detailed rules for every situation, we feed the computer a lot of data, and it figures out the patterns and relationships on its own.  Think of it as teaching a computer to “think” like a human, but with a lot more mathematical precision.\n\n**2. Key Concepts & Types**\n\n* **Data:** This is the fuel for machine learning. The more high-quality data you have, the better the model will perform.\n* **Algorithms:** These are the formulas and processes that allow the computer to learn from the data. There are many different types of 

In [39]:
agent_executor.invoke({"input" : "What is your opinion on Elon Musk?"})




> Entering new AgentExecutor chain...
Okay, let's delve into the complex topic of Elon Musk. It's a really interesting and frequently debated subject, and there's no simple answer – he's a figure of immense influence and considerable controversy. Here’s a breakdown of my perspective, combining various viewpoints and observations:

**Here's a balanced look at his key aspects:**

**What I find compelling (Reasons to appreciate him):**

* **Visionary Thinking & Risk-Taking:** Musk is undeniably a brilliant and audacious entrepreneur. He’s constantly pushing boundaries and envisioning things that others deem impossible. His ideas around space travel, electric vehicles, and autonomous systems are genuinely fascinating and could have a significant impact on humanity.
* **Innovation & Disruption:** He's inspired a wave of innovation across numerous industries. Tesla, SpaceX, and Neuralink are all examples of companies that have fundamentally altered how people interact with technology and t

{'input': 'What is your opinion on Elon Musk?',
 'output': "Okay, let's delve into the complex topic of Elon Musk. It's a really interesting and frequently debated subject, and there's no simple answer – he's a figure of immense influence and considerable controversy. Here’s a breakdown of my perspective, combining various viewpoints and observations:\n\n**Here's a balanced look at his key aspects:**\n\n**What I find compelling (Reasons to appreciate him):**\n\n* **Visionary Thinking & Risk-Taking:** Musk is undeniably a brilliant and audacious entrepreneur. He’s constantly pushing boundaries and envisioning things that others deem impossible. His ideas around space travel, electric vehicles, and autonomous systems are genuinely fascinating and could have a significant impact on humanity.\n* **Innovation & Disruption:** He's inspired a wave of innovation across numerous industries. Tesla, SpaceX, and Neuralink are all examples of companies that have fundamentally altered how people int